In [1]:
import os,sys
sys.path.append("../../")
from analysis_common import *

In [2]:
from root_common import *

Welcome to JupyROOT 6.26/00


In [3]:
#make a canvas we can use by default
c = ROOT.TCanvas()

In [4]:
#some useful functions
from analysis_helpers import *

In [5]:
#makes plotting more interactive in notebook
%jsroot on

In [6]:
#some useful early definitions
lep_accept_pt = 0.4 #GeV
lep_accept_angle = radians(40)

hadron_accept_angle = radians(40)
hadron_accept_ke = 0.060 #GeV

In [7]:
#NOTE YOU WILL NEED TO MODIFY THIS PARENT DIRECTORY!
dir_files_GEM21_11b = "/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00"
dir_files_G18_02a = "/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_G18_02a_00"

In [8]:
#create a list of the files that we have
gst_files_GEM21_11b = glob.glob(f"{dir_files_GEM21_11b}/*gst.root")
gst_files_G18_02a = glob.glob(f"{dir_files_G18_02a}/*gst.root")

In [9]:
for f in gst_files_GEM21_11b[:10]: print(f)

/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run46.gst.root
/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run47.gst.root
/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run78.gst.root
/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run79.gst.root
/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run31.gst.root
/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run30.gst.root
/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run97.gst.root
/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run96.gst.root
/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GE

In [10]:
#defined in analysis_helpers.py ... creates a TChain from our list of files
gst_chain_GEM21_11b = create_gst_chain(gst_files_GEM21_11b,verbose=True)

Created gst chain from 100 files with 10000000 total events.


In [11]:
gst_chain_G18_02a = create_gst_chain(gst_files_G18_02a,verbose=True)

Created gst chain from 100 files with 10000000 total events.


In [12]:
#creates a dataframe from our TChain
# see https://root.cern/doc/master/classROOT_1_1RDataFrame.html for examples
df_gst_GEM21_11b_all = ROOT.RDataFrame(gst_chain_GEM21_11b)

In [13]:
n_entries = df_gst_GEM21_11b_all.Count().GetValue()
print(n_entries)

10000000


In [14]:
#restrict to just 1 million events
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Range(1000000)

In [15]:
df_gst_GEM21_11b_all.Count().GetValue()

1000000

In [16]:
#apply definitions for new lepton variables
df_gst_GEM21_11b_all = define_df_gst_lep_vars(df_gst_GEM21_11b_all)

In [17]:
df_gst_GEM21_11b_all = define_df_gst_hadron_vars(df_gst_GEM21_11b_all)

In [18]:
df_gst_GEM21_11b_all = define_df_gst_hadrons_by_pdg(df_gst_GEM21_11b_all)

In [19]:
df_gst_GEM21_11b_all = define_df_gst_pi0decay(df_gst_GEM21_11b_all)

In [20]:
#Make a dataframe with default acceptances applied. Note the change in name for the dataframe!
df_gst_GEM21_11b_DefA = define_df_gst_hadron_acceptance(df_gst_GEM21_11b_all)
df_gst_GEM21_11b_DefA = define_df_gst_photon_acceptance(df_gst_GEM21_11b_DefA)
df_gst_GEM21_11b_DefA = define_df_gst_hadron_sums(df_gst_GEM21_11b_DefA,sfx=["i","f","fa"])

In [21]:
df_gst_GEM21_11b_DefA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_DefA,suffix_list=["i"],cname="i")
df_gst_GEM21_11b_DefA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_DefA,suffix_list=["f"],cname="f")
df_gst_GEM21_11b_DefA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_DefA,cname="fa_all",
                                                          suffix_list=["fa_proton","fa_neutron",
                                                                       "fa_piplus","fa_piminus",
                                                                       "fa_pi0_ph"])
df_gst_GEM21_11b_DefA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_DefA,
                                                         cname="fa_chhad",
                                                         suffix_list=["fa_proton","fa_piplus","fa_piminus"])

In [22]:
#Make a dataframe with acceptance without angular cuts applied.
df_gst_GEM21_11b_KEA = define_df_gst_hadron_acceptance(df_gst_GEM21_11b_all,thetaz_max=[10,10,10,10])
df_gst_GEM21_11b_KEA = define_df_gst_photon_acceptance(df_gst_GEM21_11b_KEA,thetaz_max=10.)
df_gst_GEM21_11b_KEA = define_df_gst_hadron_sums(df_gst_GEM21_11b_KEA,sfx=["i","f","fa"])

In [23]:
df_gst_GEM21_11b_KEA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_KEA,suffix_list=["i"],cname="i")
df_gst_GEM21_11b_KEA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_KEA,suffix_list=["f"],cname="f")
df_gst_GEM21_11b_KEA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_KEA,cname="fa_all",
                                                          suffix_list=["fa_proton","fa_neutron",
                                                                       "fa_piplus","fa_piminus",
                                                                       "fa_pi0_ph"])
df_gst_GEM21_11b_KEA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_KEA,
                                                         cname="fa_chhad",
                                                         suffix_list=["fa_proton","fa_piplus","fa_piminus"])

In [24]:
df_gst_GEM21_11b_all.Filter("ptl>0.4").Count().GetValue()

112733

In [25]:
#can print all of our columns here
for n in df_gst_GEM21_11b_DefA.GetColumnNames():
    print(n)

A
Acceptf_neutron
Acceptf_pi0_ph1
Acceptf_pi0_ph2
Acceptf_pi0_phAll
Acceptf_pi0_phNone
Acceptf_piminus
Acceptf_piplus
Acceptf_proton
DXSec
Ef
Ef_K0
Ef_Kminus
Ef_Kplus
Ef_neutron
Ef_pi0
Ef_pi0_ph1
Ef_pi0_ph2
Ef_piminus
Ef_piplus
Ef_proton
Efa_neutron
Efa_pi0_ph1
Efa_pi0_ph2
Efa_piminus
Efa_piplus
Efa_proton
Efm_neutron
Efm_pi0_ph1
Efm_pi0_ph2
Efm_piminus
Efm_piplus
Efm_proton
Ei
Ei_K0
Ei_Kminus
Ei_Kplus
Ei_neutron
Ei_pi0
Ei_piminus
Ei_piplus
Ei_proton
El
En
Ev
EvRF
KPS
Q2
Q2s
W
Ws
XSec
Z
amnugamma
calresp0
cc
charm
coh
cthf
cthl
delta_alphaf
delta_alphafa_all
delta_alphafa_chhad
delta_alphai
delta_alphatf
delta_alphatfa_all
delta_alphatfa_chhad
delta_alphati
delta_cosalphaf
delta_cosalphafa_all
delta_cosalphafa_chhad
delta_cosalphai
delta_cosalphatf
delta_cosalphatfa_all
delta_cosalphatfa_chhad
delta_cosalphati
delta_cosphif
delta_cosphifa_all
delta_cosphifa_chhad
delta_cosphii
delta_cosphitf
delta_cosphitfa_all
delta_cosphitfa_chhad
delta_cosphiti
delta_pf
delta_pfa_all
delta_pfa_chhad

In [26]:
#make a quick check to see that the proton indices are filled, and sorted by KE
rows=10
disp = df_gst_GEM21_11b_all.Range(rows).Display(["pdgf","massf","kef","kef_proton"],rows,100)
disp.Print()

+-----+------+------------+---------------+---------------+
| Row | pdgf | massf      | kef           | kef_proton    | 
+-----+------+------------+---------------+---------------+
| 0   | 2212 | 0.93827208 | 0.43649404    | 0.43649404    | 
|     | 211  | 0.13957018 | 1.8244414     | 0.053096929   | 
|     | 2112 | 0.93956541 | 0.095797126   |               | 
|     | 2112 | 0.93956541 | 0.048370546   |               | 
|     | 2112 | 0.93956541 | 0.015021265   |               | 
|     | 2112 | 0.93956541 | 0.018512205   |               | 
|     | 2212 | 0.93827208 | 0.053096929   |               | 
|     | 2112 | 0.93956541 | 0.16068890    |               | 
|     | 2112 | 0.93956541 | 0.049633567   |               | 
+-----+------+------------+---------------+---------------+
| 1   | 2212 | 0.93827208 | 0.30096254    | 0.30096254    | 
|     | 111  | 0.13497660 | 0.81738266    |               | 
|     | 111  | 0.13497660 | 1.7637369     |               | 
+-----+------+------------+

In [27]:
#define bins for a 2D plot in log-log space
etransfer_bins = array.array('d',np.logspace(np.log10(0.05),np.log10(4),100).tolist())
q2transfer_bins = array.array('d',np.logspace(np.log10(0.02),np.log10(10),100).tolist())

In [28]:
#create Q2 versus omega plot
h_lep_q2vw = df_gst_GEM21_11b_all.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,len(q2transfer_bins)-1,q2transfer_bins),
                                          "energy_transfer","Q2")

In [29]:
h_lep_q2vw.SetTitle("All events;Energy Transfer (GeV);Q^{2} (GeV/c)^{2}")
h_lep_q2vw.Draw("colz")
c.SetLogx()
c.SetLogy()
c.Draw()

In [30]:
#now, make a filter based on acceptance
df_gst_GEM21_11b_accept = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")

In [31]:
h_lep_q2vw_accept = df_gst_GEM21_11b_accept.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,
                                            len(q2transfer_bins)-1,q2transfer_bins),
                                      "energy_transfer","Q2")

In [32]:
h_lep_q2vw_accept.SetTitle("Events passing electron acceptance;Energy Transfer (GeV);Q^{2} (GeV/c)^{2}")
h_lep_q2vw_accept.Draw("colz")
c.SetLogx()
c.SetLogy()
c.Draw()

In [33]:
#reset canvas to linear scale
c.SetLogy(0)
c.SetLogx(0)

In [34]:
#Make 1D lepton pt histogram for QE, MEC..., but for those passing our lepton acceptance
h_lep_pt_QEL = df_gst_GEM21_11b_accept.Filter("qel==1").Histo1D(("","",100,0,3.0),"ptl")
h_lep_pt_MEC = df_gst_GEM21_11b_accept.Filter("mec==1").Histo1D(("","",100,0,3.0),"ptl")

In [35]:
h_lep_pt_QEL.SetLineColor(ROOT.kBlue)
h_lep_pt_MEC.SetLineColor(ROOT.kRed)
h_lep_pt_QEL.Draw()
h_lep_pt_MEC.Draw("same")
c.Draw()

In [36]:
#Make plot of KE for all protons
#we can select the protons based on the pdg code

h_kef_p_all = df_gst_GEM21_11b_all.Histo1D(("","",100,0,1.0),"kef_proton")

In [37]:
h_kef_p_all.Draw()
c.Draw()

In [38]:
#Make plot of KE for leading proton

#Now instead of plotting for all protons, we can plot for just the 'leading' (most energetic) proton
h_kef_p_max = df_gst_GEM21_11b_all.Define("kef_proton_max","Max(kef[pdgf==2212])").Histo1D(("","",100,0,1.0),"kef_proton_max")

In [39]:
h_kef_p_max.Draw()
c.Draw()

In [40]:
#Make plot of KE for leading proton, but let's remove events where the proton is very low energy (below 60 MeV)

#Now instead of plotting for all protons, we can plot for just the 'leading' (most energetic) proton
h_kef_p_max = df_gst_GEM21_11b_all.Define("kef_proton_max","Max(kef[pdgf==2212])").Filter("kef_proton_max>0.060").Histo1D(("","",100,0,1.0),"kef_proton_max")

In [41]:
h_kef_p_max.Draw()
c.Draw()